In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler as SS
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression as LR

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
df.head()

In [ ]:

spisok_without_p = ['city_development_index', 'training_hours']
s_norm = SS().fit(df[spisok_without_p])
dfn = pd.DataFrame(s_norm.transform(df[spisok_without_p]), columns=spisok_without_p)
dfn.head()

In [ ]:
# нормализация категориальных параметров
spisok_cat_p = ['gender', 'relevent_experience', 'education_level', 'experience', 'last_new_job', 'company_size', 'company_type', 'city']
for el in spisok_cat_p:
  df[el] = list(map(lambda x: str(x) , df[el]))
encoder=DV(sparse=False).fit(df[spisok_cat_p].T.to_dict().values())
converted_cat_pr=encoder.transform(df[spisok_cat_p].T.to_dict().values())
converted_cat_pr

In [ ]:
for num, el in enumerate(converted_cat_pr.T):
  dfn['p' + str(num)] = el
dfn.head()

In [ ]:
model=LR()
model.fit(dfn,df.target)
yp=model.predict_proba(dfn)
fpr, tpr, thresholds = roc_curve(df.target, yp.T[1])
plt.plot(fpr,tpr)
plt.plot([0,1],[0,1])
plt.show()
yp=model.predict_proba(dfn)
mm=thresholds[np.argmin(fpr**2+(1-tpr)**2)]
print("Порог", mm)
print ("accuracy_score", metrics.accuracy_score(df.target,np.int32(yp.T[1]>mm)))
print ("precision_score",metrics.precision_score(df.target,np.int32(yp.T[1]>mm)))
print ("recall_score", metrics.recall_score(df.target,np.int32(yp.T[1]>mm)))
print ("f1_score", metrics.f1_score(df.target,np.int32(yp.T[1]>mm)))
print ("roc_auc_score", metrics.roc_auc_score(df.target,yp.T[1]))

In [ ]:
df_test = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv')
df_test.head()

In [ ]:
dfn_test = pd.DataFrame(s_norm.transform(df_test[spisok_without_p]), columns=spisok_without_p)
dfn_test.head()

In [ ]:
for el in spisok_cat_p:
  df_test[el] = list(map(lambda x: str(x) , df_test[el]))
converted_cat_pr=encoder.transform(df_test[spisok_cat_p].T.to_dict().values())
converted_cat_pr

In [ ]:
for num, el in enumerate(converted_cat_pr.T):
  dfn_test['p' + str(num)] = el
dfn_test.head()

In [ ]:
yp=model.predict_proba(dfn_test)
print(np.int32(yp.T[1]>mm))

In [ ]:
df_res = df_test[['enrollee_id']]
df_res['target'] = yp.T[1]
df_res.head()